# Phylogenetic inference

Next, we'll make a couple quick phylogenetic trees of our samples from the ipyrad assembly. We'll use two different programs that have different theoretical underpinnings: IQ-TREE and SVDQuartets. IQ-TREE is a maximum likelhood method for estimating phylogenies on concatentated alignments. Most models in IQ-TREE and similar maximum likelihood and Bayesian phylogeny programs assume that all sites in an alignment share a single underlying tree. This assumption may be broken for many reasons, including incomplete lineage sorting and gene flow. Gene flow is still notoriously hard to detect and adequately model in phylogenetic inference, but many programs not exist that are robust to variation in phylogenetic signal across sites/genes caused by incomplete lineage sorting.

SVDQuartets is one such method. SVDQuartets uses a quartet approach that is statistically consistent with the multi-species coalescent without the computational burden of explicitly modeling the multi-spoecies coalescent process.

A full discussion of the performance of these programs in different scenarios is beyong the scope of this tutorial, but if your research involves phylogenetic inference, we highly recommend that you do some deeper reading.



<br>

## IQ-TREE

IQ-TREE is a very commonly used and very easy to use program for generating maximum likelihood phylogenies. We'll start with this.


The programs that we need for this are already in the container we've been using, but if you need to use them in other contexts you can download them at these links: [PAUP](https://paup.phylosolutions.com/get-paup/) and [IQTRee](http://www.iqtree.org/#download)


If you don't already have the ipyrad data handy, you can download my completed assembly (or your own completed assembly if you have one in a bucket but do not have it copied to your current instance):

In [1]:
! gsutil -m cp -r gs://ruber-ipyrad-out/ .

Copying gs://ruber-ipyrad-out/Localities.csv...
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.alleles...                   
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.geno...                      
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.gphocs...                    
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.loci...                      
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.migrate...                   
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.phy...                       
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.nex...                       
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.seqs.hdf5...                 
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.snps...                      
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.vcf...                       
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.snps.hdf5...                 
Copying gs://ruber-ipyrad-out/ruber_reduced_denovo.snpsmap...

It's very easy to run, we mostly just need to point iqtree to the input file, which we'll set as the phylip-formatted output from ipyrad, the `.phy` file. 


We'll set up our input and output paths as variables so that these can easily be changed and we shouldn't need to change much in the actual program call for different datasets, just these variables.

`INFILE` will be the name and path to the input file
`OUTFIX` will the be the prefix that gets prepended to each output file.
`OUTDIR` is the directory that we want all output to go into.


 Options that we'll set in the program call include:

`-s $INFILE` sets the input sequence file.

`-m MFP` which instead of specifying a model of evolution, tells IQTree to use ModelFinderPlus to find the best model of sequence evolution.

`-T auto` tells IQTree to automatically determine the best number of threads to use, within some maximum we specify based on what we've allocated.

`--prefix $OUTFIX` sets the prefix for our output to what we define in out `OUTFIX` bash variable.

`-B 1000` tells IQTree to use 1000 rapid bootstraps for assessing support.

`-alrt 1000` uses 1000 bootstrap replicates for SH-aLRT calculation (a likelihood-based metric of branch support).

`-ntmax 12` sets the maximum number of threads to use, this should not exceeed the number of cores in your instance.


In [12]:
import os



# set up the input file, outfile prefix, and output directory
os.environ["INFILE"] = "/home/jupyter/RADseq_cloud_learn/ruber-ipyrad-out/ruber_reduced_denovo.phy"
os.environ["OUTFIX"] = "ruber"
outdir = "/home/jupyter/RADseq_cloud_learn/radseq_cloud/iqtree_out"

In [13]:
os.makedirs(outdir, exist_ok=True) # create the output directory if it doesn't already exist
os.chdir(outdir)

In [15]:
## Execute iqtree

! iqtree2 -s $INFILE -m MFP -T auto --prefix $OUTFIX -B 1000 -alrt 1000 -nt AUTO -ntmax 16

IQ-TREE multicore version 2.3.6 for Linux x86 64-bit built Aug  1 2024
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor, Heiko Schmidt,
Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    b53cecafc9d7 (AVX512, FMA3, 62 GB RAM)
Command: iqtree2 -s /home/jupyter/RADseq_cloud_learn/ruber-ipyrad-out/ruber_reduced_denovo.phy -m MFP -T auto --prefix ruber -B 1000 -alrt 1000 -nt AUTO -ntmax 16
Seed:    141384 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Mon Jan 27 21:15:20 2025
Kernel:  AVX+FMA - auto-detect threads (16 CPU cores detected)

Reading alignment file /home/jupyter/RADseq_cloud_learn/ruber-ipyrad-out/ruber_reduced_denovo.phy ... Phylip format detected
Alignment most likely contains DNA/RNA sequences
Alignment has 33 sequences with 220324 columns, 9022 distinct patterns
691 parsimony-informative, 442 singleton sites, 219191 constant sites
               Gap/Ambiguity  Composition  p-value
Analyzing sequences: done in 0.00020808

If IQ-TEE runs sucessfully, your output directory should end up with various files, most importantly `ruber.treefile`. We'll visualize the tree you estimated in the R in the next tutorial

## SVDQuartets


SVDQuartets is a quartet-based method that is designed to work on SNPs to create species trees, but it can also be used with full concatenated alignments to generate trees of indiviuals like we've done with IQTree.

It is somewhat more involved to set up, and we'll again set it up with a bunch of bash variables.

What we'll do is run a single search for the best tree, save it, then run a search that includes bootstrapping and save those trees. Later, in R, we'll plot the bootstraps onto the best tree. Note that if you run a bootstrap analysis and just plot the tree that comes out from that with bootstap values at nodes, the bootstraps will be plotted on a consensus of bootstrap trees, not the tree that has the highest likelihood onyour actual data. I consider this to be highly undesirable.

### Edit the nexus file

You will need to manually edit the nexus file created by ipyrad to create a nexus file that SVDQuartets/PAUP will correctly read in. The character sets specified in the file we got from ipyrad will cause issues, and so we need to delete them. 

Use the editor to make a copy of `ruber_reduced_denovo.nex` as `ruber_reduced_denovoPAUP.nex`. Open `ruber_reduced_denovoPAUP.nex` and delete the line `BEGIN SETS;` all of the lines that begin `charset` and the `END;` that marks the end of the charsets block. Note that there are other `end;` statements in the nexus file that you do not want to delete.

Once that is done, you can proceed. We will set this up to use variables to specify the input, output, and some options for SVDQuartets so that in most cases, you should not need to edit anything in the second part of this code block. Note, however, that there are some options that we have defined in the program call, and in some cases you may want to change these.

In [2]:
%%bash
PAUP="/usr/bin/paup4" # set up PAUP path
OUTDIR="/home/jupyter/RADseq_cloud_learn/radseq_cloud/svdq_out"


#define  variables for the PAUPblock
filebname="ruber_reduced_denovo" #basename for all produced files
infile="/home/jupyter/RADseq_cloud_learn/ruber-ipyrad-out/ruber_reduced_denovoPAUP.nex" #name of input nexus file; can give a path so the input files don't have to be part of the working directory
nthreads=16 #number of threads to use
nreps=200 #number of replicates for bootstrapping



################################################################################################################################################################
################################################################################################################################################################
####    Run based on the parameters set above
################################################################################################################################################################
################################################################################################################################################################


#change working directory to where your output files will go
mkdir -p $OUTDIR
cd $OUTDIR


cat <<EOF > $filebname.paup.txt
Begin paup;
set autoclose=yes warntree=no warnreset=no flock=no;
log start file=$filebname.log ;
execute $infile;
svdQuartets evalQuartets=all showScores=no ambigs=distribute bootstrap=no nthreads=$nthreads;
savetrees file=$filebname.besttree.tre;
svdQuartets evalQuartets=all showScores=no ambigs=distribute bootstrap=standard nreps=$nreps nthreads=$nthreads treefile=$filebname.svdqboots.tre;  
quit; 
end;
EOF

$PAUP $filebname.paup.txt #execute your new paup block file






P A U P *
Version 4.0a (build 169) for Unix/Linux (built on May 28 2024 at 11:04:55)
Wed Jan 29 23:05:19 2025

        -----------------------------NOTICE-----------------------------
          This is a test version that is still changing rapidly.
          Please report bugs to dave@phylosolutions.com
        ----------------------------------------------------------------

Running on Intel(R) Xeon(R) CPU @ 2.80GHz
    Current processor contains 8 CPU cores on 1 socket (hyperthreaded to 16 logical cores)
    Executable built for Intel(R) 64 architecture
    Compiled using GNU C compiler (gcc) 7.5.0
    SSE vectorization enabled
    SSSE3 instructions supported
    Multithreading enabled using Pthreads

Processing of file "~/RADseq_cloud_learn/radseq_cloud/svdq_out/ruber_reduced_denovo.paup.txt" begins...

        -----------------------------NOTICE-----------------------------
          This is a test version that is still changing rapidly.
          Please report bugs to dave@phylo


Getting SVD scores for all quartets...
[##################################################] 100.0% 



******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  



Assembling quartets...
[##################################################] 100.0% 



Quartet assembly completed:
  Total weight of incompatible quartets = 5732 (14.008%)
  Total weight of compatible quartets   = 35188 (85.992%)
  Time used for QFM = 0.07 sec (CPU time = 1.01 sec)

Tree from SVDQuartets analysis (also stored to tree buffer):

/--------------------------------------------------------------------------------------------------------------------------------------- SD Field 0201
|
|                                                                                                                               /------- SD Field 0255
|                                                                                                                         /-----+
|                                                                                                                         |     \------- SD Field 2914
|                                                                                                                  /------+
|                              

...


Quartet assembly completed:
  Total weight of incompatible quartets = 5732 (14.008%)
  Total weight of compatible quartets   = 35188 (85.992%)
  Time used for QFM = 0.07 sec (CPU time = 0.97 sec)

Tree from SVDQuartets analysis (also stored to tree buffer):

/--------------------------------------------------------------------------------------------------------------------------------------- SD Field 0201
|
|                                                                                                                               /------- SD Field 0255
|                                                                                                                         /-----+
|                                                                                                                         |     \------- SD Field 2914
|                                                                                                                  /------+
|                              


Bootstrapping (SVDQuartets)...



******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  

******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  

******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  

******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  

******  
******  Warning: Requesting more threads (16) than the number of available logical cores (16) on hyperthreaded systems may strongly degrade
******           performance!
******  

******  
******  Warning: Requesting more threads (16)

[##################################################] 100.0% 



Bootstrap 50% majority-rule consensus tree
   (plus other groups compatible with this tree)

/----------------------------------------------------------------------------------------------------------------------------------- SD Field 0201(1)
|
|                                                                                                                            /------ SD Field 0255(2)
|                                                                                                               /-----37-----+
|                                                                                                               |            \------ SD Field 2914(30)
|                                                                                                         /-10--+
|                                                                                                         |     |      /------------ SD Field 1225(16)
|                                                            

If SVDQuartets runs sucessfully, your output directory should end up with various files, most importantly `ruber_reduced_denovo.svdqboots.tre` and `ruber_reduced_denovo.besttree.tre`. We'll visualize the tree you estimated in the R in the next tutorial.